In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd


headers = ({'User-Agent':
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit\
            /537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'})
data_list = []
for page in range(1,21):
    base_url = "https://www.autoscout24.de/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=" +str(page) + "&cy=D&body=4&atype=C&fc=1&qry=&"
    response = get(base_url,headers=headers)
    #print(response)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    data_list.append(soup.find_all('div', attrs={'class': 'cldt-summary-full-item-main'}))
#print(data_list)

In [31]:
model_info = []
price_info = []
general_info = []
for item in data_list:
    for auto in item:
        model_info.append(auto.find_all('div', attrs={'class': 'cldt-summary-title'}))
        price_info.append(auto.find_all('div', attrs={'class': 'cldt-summary-payment'}))
        general_info.append(auto.find_all('div', attrs={'class': 'cldt-summary-vehicle-data'}))
    
#print(model_info)
#print(price_info)
#print(general_info)

In [75]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [94]:
brands = []
models = []
for model in model_info:
    for m in model:
        md = m.find_all("h2", attrs={'class': "cldt-summary-makemodel sc-font-bold sc-ellipsis"})[0].text.strip()
        if len(md.split()) >= 3:
            if 'Land Rover' in md:
                brands.append("Land Rover")
                models.append(md.split()[2])
            else:        
                brands.append(md.split()[0])
                temp = md.split()[1:]
                mdl = convert_list_to_string(temp)
                models.append(mdl)
            
                
        
        else:
            brands.append(md.split()[0])
            models.append(md.split()[1])
#print(brands)
#print(models)

In [95]:
prices = []
for price in price_info:
    for p in price:
        pr = p.find_all("span", attrs={"class": "cldt-price sc-font-xl sc-font-bold"})[0].text.strip()
        prc = pr.split()[1].replace("-","").replace(",", "").replace(".","")
        prices.append(float(prc))
#print(prices)

In [96]:
first_reg_year = []
milleage = []
transmission = []
fuel_type = []
engine_power = []
        
for auto in general_info:
    for spec in auto:
        prod = spec.find_all('li')[1].text.strip()
        first_reg_year.append(prod)
                         
        ml = spec.find_all('li')[0].text.strip().replace(".", "").split()[0]
        if ml == "-":
            ml = '100'
            milleage.append(ml)
            
        else:
            milleage.append(ml)
        
        fuel_type.append(spec.find_all('li', attrs={'class': "summary_item_no_bottom_line"})[0].text.strip())
        transmission.append(spec.find_all('li')[5].text.strip())
        
        enp = spec.find_all('li')[2].text.strip().replace("(","").replace(")","").split()[2]
        engine_power.append(float(enp))
      
#print(fuel_type)
#print(prod_year)
#print(milleage)
#print(engine_power)
#print(transmission)


In [102]:
data = pd.DataFrame({'Brand': brands, "Model": models, "Fuel type": fuel_type, "Transmission": transmission, "First Registration Year": first_reg_year, "Milleage (kms)":milleage, "Engine Power (PS)": engine_power, "Price (EUR)":prices})

In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Brand                    400 non-null    object 
 1   Model                    400 non-null    object 
 2   Fuel type                400 non-null    object 
 3   Transmission             400 non-null    object 
 4   First Registration Year  400 non-null    object 
 5   Milleage (kms)           400 non-null    object 
 6   Engine Power (PS)        400 non-null    float64
 7   Price (EUR)              400 non-null    float64
dtypes: float64(2), object(6)
memory usage: 25.1+ KB


In [104]:
data.describe()

,Engine Power (PS),Price (EUR)
count,400.000000,400.000000
mean,131.330000,11268.597500
std,37.848744,2151.021892
min,75.000000,1490.000000
25%,110.000000,9990.000000
50%,125.000000,11899.500000
75%,140.000000,12990.000000
max,349.000000,13790.000000


In [105]:
data.to_csv('Car_list.csv', index=False)